## ME5413: Autonomous Mobile Robot  

### Homework 1: Perception  
Due date: 22 February 2024 (Thurs) - 2359 


### Task 1.1 Single-Object Tracking 


Using Template Matching


In [1]:
## put your code here
import cv2
import numpy as np
import math
# by changing the flag, you can check the output of five seq bag
flag = 1 #1,2,3,4,5

# read first_track.txt
with open(f'../data/Task 1/seq_{flag}/firsttrack.txt', 'r') as file: 
    line = file.readline()
    template_coordinates = [int(coord) for coord in line.split(',')]

# read ground_true.txt
with open(f'../data/Task 1/seq_{flag}/groundtruth.txt', 'r') as file:
    ground_truths = [list(map(int, line.split(','))) for line in file]

# template figure
template_image_path = f'../data/Task 1/seq_{flag}/img/001.jpg'
template_image = cv2.imread(template_image_path)  


# image process
white_threshold = 200  
mask = cv2.inRange(template_image, (white_threshold, white_threshold, white_threshold), (255, 255, 255))
template_image[mask == 255] = [0, 0, 0]


x, y, w, h = template_coordinates
template = template_image[y:y+h,x:x+w]

# match result contains the (x,y) coordinates of the tracking points
match_result = []

# make template matching
for i in range(1,101):
    image_path = f'../data/Task 1/seq_{flag}/img/{str(i).zfill(3)}.jpg'
    img = cv2.imread(image_path)

    #image process
    white_threshold = 200
    mask = cv2.inRange(img, (white_threshold, white_threshold, white_threshold), (255, 255, 255))
    img[mask == 255] = [0, 0, 0]
 
    # doing template matching
    result = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)

    _, _, _, max_loc = cv2.minMaxLoc(result)

    match_result.append(max_loc)

    #ground true 
    x_g, y_g, w_g, h_g = ground_truths[i - 1]
    #draw ground true bounding box
    cv2.rectangle(img, (x_g, y_g), (x_g + w_g, y_g + h_g), (0, 0, 255), 2)

    # # Get the width and height of the template
    template_h, template_w, _ = template.shape

    # # Draw a rectangle around the matched region
    cv2.rectangle(img, max_loc, (max_loc[0] + template_w, max_loc[1] + template_h), (255,0,0), 2)

    # # update template
    # template = img[max_loc[1]:max_loc[1] + template_h, max_loc[0]:max_loc[0] + template_w].copy()

    # Display the result
    cv2.imshow('Result', img)
    # save the image
    # cv2.imwrite(f'fig_save/seq_{flag}_template/result_image{i-1}.jpg', img)
    cv2.waitKey(200)
    cv2.destroyAllWindows()
    
# print('match_result:',match_result)

# match_result area contains the size of the bounding box
match_result_area = np.array([(template_w, template_h)] * len(match_result))
ground_truths_coordinate = [(item[0], item[1]) for item in ground_truths]

# write the results to .txt file
# output = np.hstack((match_result,match_result_area))
# with open(f'template_output_seq{flag}.txt', 'w') as file:
#     for item in output:
#         file.write(f'{item[0]}, {item[1]}, {item[2]}, {item[3]}\n')

print('ground_truths_coordinate:',ground_truths_coordinate)


ground_truths_coordinate: [(173, 294), (175, 294), (173, 294), (172, 296), (172, 295), (171, 295), (173, 295), (171, 296), (171, 295), (169, 296), (171, 294), (173, 295), (172, 295), (172, 294), (174, 294), (175, 294), (175, 292), (177, 292), (178, 289), (181, 287), (184, 286), (185, 287), (185, 287), (186, 289), (186, 288), (184, 288), (184, 288), (184, 291), (183, 290), (183, 290), (184, 291), (184, 292), (182, 292), (183, 295), (182, 294), (177, 293), (180, 292), (183, 290), (188, 289), (192, 286), (222, 287), (235, 287), (251, 290), (257, 291), (263, 292), (278, 292), (285, 289), (294, 287), (313, 285), (334, 286), (345, 283), (344, 283), (353, 281), (359, 280), (369, 280), (368, 276), (407, 277), (427, 277), (419, 276), (434, 274), (446, 272), (457, 272), (465, 271), (478, 271), (501, 270), (509, 271), (511, 268), (512, 264), (509, 264), (519, 263), (530, 262), (545, 262), (551, 263), (551, 264), (554, 262), (556, 260), (559, 256), (561, 257), (568, 258), (573, 261), (575, 262), (

Using Kalman Filter


In [2]:
import cv2
import numpy as np

# by changing the flag, you can check the output of five seq bag
flag = 4 #1,2,3,4,5

# read first_track.txt
with open(f'../data/Task 1/seq_{flag}/firsttrack.txt', 'r') as file:
    line = file.readline()
    template_coordinates = [int(coord) for coord in line.split(',')]

# read ground_true
with open(f'../data/Task 1/seq_{flag}/groundtruth.txt', 'r') as file:
    ground_truths = [list(map(int, line.split(','))) for line in file]   # template figure

template_image_path = f'../data/Task 1/seq_{flag}/img/001.jpg'
template_image = cv2.imread(template_image_path, cv2.IMREAD_COLOR)
x, y, w, h = template_coordinates
template = template_image[y:y+h, x:x+w]

# set Kalman Filter
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.array([[1, 0, 0, 0],
                                   [0, 1, 0, 0],
                                   [0, 0, 1, 0],
                                   [0, 0, 0, 1]], np.float32) * 10

# initilize the Kalman Filter state
kalman.statePre = np.array([[x + w / 2], [y + h / 2], [0], [0]], np.float32)

# match_result area contains the size of the bounding box
match_result_area = []
# match result contains the (x,y) coordinates of the tracking points
match_result = []

for i in range(1,101):
    image_path = f'../data/Task 1/seq_{flag}/img/{str(i).zfill(3)}.jpg'
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # use Kalman Filter to predict
    prediction = kalman.predict()

    #ground true bounding box
    x_g, y_g, w_g, h_g = ground_truths[i - 1]

    cv2.rectangle(img, (x_g, y_g), (x_g + w_g, y_g + h_g), (255, 0, 0), 2)

    # correct the state of Kalman Filter
    measurement = np.array([[x_g + w_g / 2], [y_g + h_g / 2]], np.float32)
    kalman.correct(measurement)

    match_result_area.append((w_g, h_g))
    match_result.append((int(prediction[0] - w_g / 2), int(prediction[1] - h_g / 2)))
    
    # draw the results
    cv2.rectangle(img, (int(prediction[0] - w_g / 2), int(prediction[1] - h_g / 2)),
                  (int(prediction[0] + w_g / 2), int(prediction[1] + h_g / 2)), (0, 255, 0), 2)
    
    # write the results to .txt file
    # output = np.hstack((match_result,match_result_area))              
    # with open(f'kalmanfilter_output_seq{flag}.txt', 'w') as file:
    #     for item in output:
    #         file.write(f'{item[0]}, {item[1]}, {item[2]}, {item[3]}\n')

    # Display the result
    cv2.imshow('Result', img)
    cv2.waitKey(500)
    cv2.destroyAllWindows()


Evaluate the performance of the Single object tracking algorithm.

template:

"""Accuracy""" 1: 53% 2: 37% 3: 56%  4:7%   5:30%

Kalman Filter:

"""Accuracy""" 1: 96% 2: 98% 3: 97%  4:97%   5:96%

In [3]:
"""accuracy"""
ground_truths = np.array(ground_truths)

distance = [math.sqrt((x1 - x2)**2 + (y1 - y2)**2) for (x1, y1),(x2,y2) in zip(match_result,ground_truths[:,0:2])]

# Define a threshold value
threshold = 20

# Count the number of accurate distances
accurate_distances_count = sum(1 for dis in distance if dis < threshold)

# Calculate accuracy
accuracy = accurate_distances_count / len(distance)

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 97.00%


template 

"""Success Rate""" 
1: 58.9%    2: 50.9%    3: 54.2%    4: 41.7%     5: 25.3%

kalman filter
1: 88.5%    2: 94.0%    3: 88.7%    4: 96.5%     5: 89.2%

In [4]:
"""Success"""
# Define a function to calculate overlap area between two rectangles
def overlap_area(rect1, rect2):
    x1, y1, w1, h1 = rect1
    x2, y2, w2, h2 = rect2
    
    overlap_x1 = max(x1, x2)
    overlap_y1 = max(y1, y2)
    overlap_x2 = min(x1 + w1, x2 + w2)
    overlap_y2 = min(y1 + h1, y2 + h2)
    
    overlap_width = max(0, overlap_x2 - overlap_x1)
    overlap_height = max(0, overlap_y2 - overlap_y1)
    
    overlap_area = overlap_width * overlap_height
    
    return overlap_area

overlap_success = []

for i in range(len(ground_truths)):
    # prediction bounding box
    rect1 = np.hstack((match_result[i], match_result_area[i]))
    # ground truth bounding box
    rect2 = ground_truths[i]

    area = overlap_area(rect1, rect2)
    success = area / (rect1[2] * rect1[3] + rect2[2] * rect2[3] - area)
    overlap_success.append(success)
print('overlap_success =', overlap_success)

Average_success_rate = np.average(overlap_success)
print('success rate =', Average_success_rate)


overlap_success = [0.0, 0.6126456296469762, 0.8373575193038363, 0.8757984796369628, 0.9502058478829222, 0.9765921683371723, 0.9718599862731641, 0.9716751310954209, 0.9960707269155207, 0.9920948616600791, 0.9960238568588469, 0.9517865703012648, 0.9866377341251713, 1.0, 0.9958592132505176, 1.0, 0.9715951562266407, 0.9913793103448276, 0.9642857142857143, 0.9890710382513661, 0.9806159505730321, 0.9509248210023866, 0.9753124517821324, 0.9791247621636314, 0.975006913261611, 0.9831746167314345, 0.9617773943054357, 0.935106610568352, 0.9580235210474864, 0.987012987012987, 0.9695921958447813, 0.9695921958447813, 0.9676383799734103, 0.9767905832280375, 0.9681194444904242, 0.9727891156462585, 0.9717789748280652, 0.9675222986314428, 0.9659115426105718, 0.9851851851851852, 0.9855072463768116, 0.9807345611368377, 0.9951219512195122, 0.9951219512195122, 0.966460249847872, 0.98068175640166, 0.9949238578680203, 0.9665302543116048, 0.9897435897435898, 0.9617965926690759, 0.9755114230252352, 0.9667610672

template
"""P_norm"""  
1: 0.998  2: 0.496  3: 0.491  4: 0.457  5: 2.34

kalman filter
1: 0.076  2: 0.051  3: 0.152  4: 0.034  5: 0.088

In [5]:
"""P_norm"""
ground_truths = np.array(ground_truths)
distance = [math.sqrt(((x1 - x2)/template_width)**2 + ((y1 - y2)/template_height)**2) 
            for (x1, y1),(x2, y2),(template_width,template_height) in zip(match_result,ground_truths[:,0:2],match_result_area)]
print(distance)
P_norm = np.average(distance)
print('P_norm =', P_norm)

[2.2832833744595162, 0.20933767385043606, 0.07434304307574847, 0.05635488616870073, 0.02370296939623307, 0.008831767886456707, 0.010277609719747583, 0.01035886258173893, 0.001968503937007874, 0.003968253968253968, 0.00199203187250996, 0.01972560854440341, 0.005119219994089748, 0.0, 0.002074688796680498, 0.0, 0.011052339753636026, 0.004329004329004329, 0.01618940122642837, 0.005494505494505495, 0.006973159640077324, 0.02309518982732583, 0.008891916061165763, 0.007543104843665977, 0.009008138253057511, 0.006596383584613677, 0.014391794055645377, 0.031338873268675266, 0.019255347561542523, 0.006535947712418301, 0.013290043079135574, 0.013290043079135574, 0.0118674007891051, 0.00844735376248257, 0.014007089016317564, 0.013793103448275862, 0.010160408282529286, 0.014297209507348108, 0.01512527910756255, 0.007462686567164179, 0.0072992700729927005, 0.00769787690332115, 0.0024449877750611247, 0.0024449877750611247, 0.01481000476677977, 0.007673778548531644, 0.002544529262086514, 0.01471783032


Visualise the results as well. 


In [6]:
## the visualisation of the results is shown above by cv2.imshow()

Propose Improvements to the work if possible: